In [1]:
import mysql.connector
import pandas as pd
from sqlalchemy import create_engine

In [2]:
# get user and password
user = ''
password = ''
with open('../user.txt', 'r') as f:
    user = f.readline().strip()
    password = f.readline().strip()
    f.close()

In [3]:
# try to connect to mysql server
mydb = mysql.connector.connect(
  host="localhost",
  user=user,
  password=password
)

print(mydb)

In [4]:
# check databases

mycursor = mydb.cursor()

# show database
sql = "SHOW DATABASES"
mycursor.execute(sql)

database = 'mydatabase'
exists = False
for x in mycursor:
    if database in x:
        exists = True
    print(x)
print(f'database "{database}" exists: {exists}')

('information_schema',)
('mydatabase',)
('mysql',)
('performance_schema',)
('sys',)
database "mydatabase" exists: True


In [5]:
# create database
if not exists:
    sql = f"CREATE DATABASE {database}"
    mycursor.execute(sql)

    # show database
    sql = "SHOW DATABASES"
    mycursor.execute(sql)

    exists = False
    for x in mycursor:
        if database in x:
            exists = True
        print(x)
    print(f'database "{database}" exists: {exists}')

In [6]:
# show table
mydb = mysql.connector.connect(
  host="localhost",
  user=user,
  password=password,
  database=database
)

table_train_name = 'house_train_data'
table_train_exist = False
table_test_name = 'house_test_data'
table_test_exist = False

mycursor = mydb.cursor()

sql = "SHOW TABLES"
mycursor.execute(sql)

for x in mycursor:
    if table_train_name in x:
        table_train_exist = True
    if table_test_name in x:
        table_test_exist = True
    print(x)

In [7]:
"""
help functions to create mysql table
"""

'''
Load a csv file into a dataframe; if csv does not have headers, use the headers arg to create a list of headers; rename unnamed columns to conform to mysql column requirements
'''
def csv_to_df(infile, headers = []):
    if len(headers) == 0:
        df = pd.read_csv(infile)
    else:
        df = pd.read_csv(infile, header = None)
        df.columns = headers
    for r in range(10):
        try:
            df.rename( columns={'Unnamed: {0}'.format(r):'Unnamed{0}'.format(r)},    inplace=True )
        except:
            pass
    return df

'''
Create a mapping of df dtypes to mysql data types (not perfect, but close enough)
'''
def dtype_mapping():
    return {'object' : 'TEXT',
        'int64' : 'INT',
        'float64' : 'FLOAT',
        'datetime64' : 'DATETIME',
        'bool' : 'TINYINT',
        'category' : 'TEXT',
        'timedelta[ns]' : 'TEXT'}
'''
Create a sqlalchemy engine
'''
def mysql_engine(user = 'root', password = 'abc', host = '127.0.0.1', port = '3306', database = 'mydatabase'):
    engine = create_engine(f'mysql+mysqlconnector://{user}:{password}@{host}:{port}/{database}', echo=False)
    return engine

'''
Create a mysql connection from sqlalchemy engine
'''
def mysql_conn(engine):
    conn = engine.raw_connection()
    return conn
'''
Create sql input for table names and types
'''
def gen_tbl_cols_sql(df):
    dmap = dtype_mapping()
    sql = "pi_db_uid INT AUTO_INCREMENT PRIMARY KEY"
    df1 = df.rename(columns = {"" : "nocolname"})
    hdrs = df1.dtypes.index
    hdrs_list = [(hdr, str(df1[hdr].dtype)) for hdr in hdrs]
    for i, hl in enumerate(hdrs_list):
        sql += " ,{0} {1}".format(hl[0], dmap[hl[1]])
    return sql

'''
Create a mysql table from a df
'''
def create_mysql_tbl_schema(df, conn, db, tbl_name):
    tbl_cols_sql = gen_tbl_cols_sql(df)
    sql = "USE {0}; CREATE TABLE {1} ({2})".format(db, tbl_name, tbl_cols_sql)
    cur = conn.cursor()
    cur.execute(sql, multi=True)
    cur.close()
    conn.commit()

'''
Write df data to newly create mysql table
'''
def df_to_mysql(df, engine, tbl_name):
    df.to_sql(tbl_name, engine, if_exists='replace')


'''
Create tables from dataframe
'''
def create_table(infile, user, password, database, table_name):
    df = csv_to_df(infile)
    engine = mysql_engine(user=user,password=password,database=database)
    create_mysql_tbl_schema(df, mysql_conn(engine), database, table_name)
    df_to_mysql(df, engine, table_name)

In [8]:
# create table

if not table_train_exist:
    infile = './kaggle-house-prices/train.csv'
    create_table(infile, user, password, database, table_train_name)

if not table_test_exist:
    infile = './kaggle-house-prices/test.csv'
    create_table(infile, user, password, database, table_test_name)
